In [24]:
import polars as pl
import os
import numpy as np
import torch, torchvision
from torchvision.utils import make_grid
from glio.loaders import nifti
from glio.python_tools import get_all_files, type_str
from glio.jupyter_tools import clean_mem
from glio.visualize import Visualizer
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq

Рассматриваем изображения *`E:\dataset\PKG - UPENN-GBM-NIfTI\UPENN-GBM\NIfTI-files\images_DTI\images_DTI\UPENN-GBM-*\UPENN-GBM-*_DTI_AD.nii.gz`*

Размер - **[240, 240, 155]**

Можно удалить пустые разрезы по каждому измерению - **[44:191, 36:222, 2:141]**

Тогда получаем изображения размера **[147, 186, 139]**, суммарно 3,800,538 значений.

Все изображения можно загрузить в память - занимается 28/40 ГБ.

Изображения имеют следующие характеристики: *min = 0; max ~= 0.003-0.005; mean ~= 0.0003-0.0007; std ~= 0.0006-0.0008*.

Значения для нормализации:
```python
mean = [0.0002149488]
std = [0.0005426282]
```

In [27]:
path = r"E:\dataset\PKG - UPENN-GBM-NIfTI\UPENN-GBM"
table = pq.read_table(f"{path}/UPENN_GBM.parquet")
with open("UPENN_GBM column names.txt", "w") as f: f.write("\n".join(list(reversed(table.column_names))))
with open("UPENN_GBM column dtypes.csv", "w") as f:
    lines = ["column,dtype,sample"]
    for i, col in enumerate(table.column_names):
        lines.append(f"{col},{type_str(table[i][0])},{str(table[i][0])}")
    f.write("\n".join(list(reversed(lines))))
with open("UPENN_GBM DF cols.csv", "w") as f:
    lines = [f"column,dtype,{','.join([str(x) for x in range(671)])}"]
    for i, col in enumerate(table.column_names):
        lines.append(f"{col},{type_str(table[i][0])},{','.join([str(table[i][j]) for j in range(671)])}")
    f.write("\n".join(list(reversed(lines))))

### Полезные колонки
- ID: str
- Survival_from_surgery_days_UPDATED: int

#### Пути к файлам
- PATH_segm.nii.gz
- PATH_T2_unstripped.nii.gz
- PATH_T1_unstripped.nii.gz
- PATH_T1GD_unstripped.nii.gz
- PATH_FLAIR_unstripped.nii.gz
- PATH_T2.nii.gz
- PATH_T1GD.nii.gz
- PATH_T1.nii.gz
- PATH_FLAIR.nii.gz
- PATH_DTI_TR.nii.gz
- PATH_DTI_RD.nii.gz
- PATH_DTI_FA.nii.gz
- PATH_DTI_AD.nii.gz
- PATH_DSC_PSR.nii.gz
- PATH_DSC_PH.nii.gz
- PATH_DSC_ap-rCBV.nii.gz
- PATH_DSC.nii.gz
- PATH_automated_approx_segm.nii.gz